# Data Collection - Using API Visual Crossing

### Import các thư viện cần thiết

In [6]:
import pandas as pd
import requests
import bs4 as BeautifulSoup
import json
from dotenv import load_dotenv
import os

### Thiết lập API và danh sách khu vực cần thu thập dữ liệu thời tiết

In [ ]:
# Lưu api key vào file .env để bảo mật
load_dotenv()
api_key = os.getenv('API_KEY')
# url mặc định
base_url = 'https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/'
# khu vực đông nam bộ
locations = ['Son La Viet Nam','Lang Son',
            'Ha Noi', 'Nghe An', 'Da Nang', 'Lam Dong',
            'Ho Chi Minh','Ben Tre']

# Do mỗi ngày chỉ crawl được khoảng 1000 ngày nên em chia ra 1 khu vực crawl trong 2 ngày
# Nửa đầu
date1 = '2019-01-01'
date2 = '2021-06-30'
# Nửa sau
date3 = '2021-07-01'
date4 = '2024-01-01'

### Gọi API để lấy dữ liệu thời tiết

In [ ]:
def getData(location: str, date1: str, date2: str) -> dict:
    '''Hàm lấy dữ liệu thời tiết từ Visual Crossing API.'''
    url = f"{base_url}{location}/{date1}/{date2}"
    params = {
        "key": api_key,
        "unitGroup": "metric",  # Sử dụng độ C
        "include": "days",   # Lấy dữ liệu thời tiết hiện tại
        "contentType": "json",  # Định dạng trả về là JSON
        "lang" : "en"
    }
    response = requests.get(url, params=params)
    # Nếu status code khác 200 -> lỗi khi gọi api
    assert response.status_code == 200, f'Error: {response.status_code}\n{response.content}'
    return response.json()

def saveJson(data: dict, fileName: str):
    '''Lưu file json phòng trường hợp cần dùng lại mà không cần phải gọi lại API'''
    with open('../khac/' + fileName, 'w') as f:
        f.write(str(data).replace('\'', '"') # đổi các cặp ngoặc '' thành "" 
                         .replace('None', '""')) # đổi các mục không có data từ None thành ""

def readJson(fileName: str):
    '''Đọc file json'''
    with open('../khac/' + fileName, 'r') as f:
        return json.load(f)

### Chuyển đổi định dạng Json sang Dataframe cho từng tỉnh thành

In [ ]:
def up1stChar(s: str) -> str:
    '''Hàm ghi hoa ký tự đầu'''
    return s[0].upper() + s[1:]

def convertDataToDf(data: dict) -> pd.DataFrame:
    '''Hàm chuyển data file json thành một Dataframe'''
    df_header = list(data['days'][0].keys())
    dict_data = {
        up1stChar(header): [] for header in df_header
    }
    for d in data['days']:
        for header in df_header:
            dict_data[up1stChar(header)].append(str(d[header])
                                        .replace('[', '')
                                        .replace(']', '')
                                        .replace('\'', '')) # bỏ các dấu ', [, ] trong list khi chuyển qua str
    # dict_data['Address'] = [data['address']] * len(data['days'])
    df = pd.DataFrame(dict_data)
    return df

def mergeDF(index) -> pd.DataFrame:
    '''Hàm merge các file json của một tỉnh thành thành một Dataframe'''
    oldData = readJson(f'../khac/{locations[index]} (1).json')
    data = readJson(f'../khac/{locations[index]} (2).json')

    df1 = convertDataToDf(oldData)
    df2 = convertDataToDf(data)

    df = pd.concat([df1, df2], axis=0, ignore_index=True)
    return df

### Thu thập dữ liệu thời tiết của 8 tỉnh thành

#### Date 1 -> Date 2 của 8 tỉnh thành

In [ ]:
# Sơn La
data = getData(locations[0], date1, date2)
saveJson(data, f'{locations[0]} (1).json')

In [ ]:
# Lạng Sơn
data = getData(locations[1], date1, date2)
saveJson(data, f'{locations[1]} (1).json')

In [ ]:
# Hà Nội
data = getData(locations[2], date1, date2)
saveJson(data, f'{locations[2]} (1).json')

In [ ]:
# Nghệ An
data = getData(locations[3], date1, date2)
saveJson(data, f'{locations[3]} (1).json')

In [ ]:
# Đà Nẵng
data = getData(locations[4], date1, date2)
saveJson(data, f'{locations[4]} (1).json')

In [ ]:
# Lâm Đồng
data = getData(locations[5], date1, date2)
saveJson(data, f'{locations[5]} (1).json')

In [ ]:
# Hồ Chí Minh
data = getData(locations[6], date1, date2)
saveJson(data, f'{locations[6]} (1).json')

In [ ]:
# Bến Tre
data = getData(locations[7], date1, date2)
saveJson(data, f'{locations[7]} (1).json')

#### Date 3 -> Date 4 của 8 tỉnh thành 

In [ ]:
# Sơn La
data = getData(locations[0], date3, date4)
saveJson(data, f'{locations[0]} (2).json')

In [ ]:
# Lạng Sơn
data = getData(locations[1], date3, date4)
saveJson(data, f'{locations[1]} (2).json')

In [ ]:
# Hà Nội
data = getData(locations[2], date3, date4)
saveJson(data, f'{locations[2]} (2).json')

In [ ]:
# Nghệ An
data = getData(locations[3], date3, date4)
saveJson(data, f'{locations[3]} (2).json')

In [ ]:
# Đà Nẵng
data = getData(locations[4], date3, date4)
saveJson(data, f'{locations[4]} (2).json')

In [ ]:
# Lâm Đồng
data = getData(locations[5], date3, date4)
saveJson(data, f'{locations[5]} (2).json')

In [ ]:
# Hồ Chí Minh
data = getData(locations[6], date3, date4)
saveJson(data, f'{locations[6]} (2).json')

In [ ]:
# Bến Tre
data = getData(locations[7], date3, date4)
saveJson(data, f'{locations[7]} (2).json')

### Gộp dữ liệu của 8 tỉnh thành cho 2 lần thu thập dữ liệu

In [ ]:
LOCATION_FILE_NAME = ['SonLa','LangSon',
            'HaNoi', 'NgheAn', 'DaNang', 'LamDong',
            'HoChiMinh','BenTre']

In [ ]:
# Sơn La
df = mergeDF(0)
df.to_csv(f'../../data/raw_data/{LOCATION_FILE_NAME[0]}.csv', index=0)

# Lạng Sơn
df = mergeDF(1)
df.to_csv(f'../../data/raw_data/{LOCATION_FILE_NAME[1]}.csv', index=0)

In [ ]:
# Hà Nội
df = mergeDF(2)
df.to_csv(f'../../data/raw_data/{LOCATION_FILE_NAME[2]}.csv', index=0)

# Nghệ An
df = mergeDF(3)
df.to_csv(f'../../data/raw_data/{LOCATION_FILE_NAME[3]}.csv', index=0)

In [ ]:
# Đà Nẵng
df = mergeDF(4)
df.to_csv(f'../../data/raw_data/{LOCATION_FILE_NAME[4]}.csv', index=0)

# Lâm Đồng
df = mergeDF(5)
df.to_csv(f'../../data/raw_data/{LOCATION_FILE_NAME[5]}.csv', index=0)

In [ ]:
# Hồ Chí Minh
df = mergeDF(6)
df.to_csv(f'../../data/raw_data/{LOCATION_FILE_NAME[6]}.csv', index=0)

# Bến Tre
df = mergeDF(7)
df.to_csv(f'../../data/raw_data/{LOCATION_FILE_NAME[7]}.csv', index=0)